In [1]:
import numpy as np
import pandas as pd
import scipy

from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [2]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [3]:
#Task 3

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [4]:
totalTest = 34
yawnTest = 10

totalControl = 16
yawnControl = 4

testSet = np.concatenate((np.zeros(totalTest - yawnTest), np.ones(yawnTest)), axis=0)
controlSet = np.concatenate((np.zeros(totalControl - yawnControl), np.ones(yawnControl)), axis=0)

stat = proportions_diff_z_stat_ind(testSet, controlSet)
p_value = proportions_diff_z_test(stat, 'greater')


print "testSet.sum = ", testSet.sum(), "testSet.len = ", len(testSet)
print "controlSet.sum = ", controlSet.sum(), "controlSet.len = ", len(controlSet)
print "Stat = ", stat, "p_value = ", p_value

testSet.sum =  10.0 testSet.len =  34
controlSet.sum =  4.0 controlSet.len =  16
Stat =  0.324101861776 p_value =  0.372930458725


Task 4

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X1,X2,X3;
логистическая регрессия по признакам X4,X5,X6.
Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [8]:
#Независимые выборки
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
#Зависимые выборки

def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [9]:
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

data = pd.read_csv("banknotes.txt", sep="\t");
Y = data.real.as_matrix()
X_1_3 = data.as_matrix(["X1", "X2", "X3"])
X_4_6 = data.as_matrix(["X4", "X5", "X6"])
X_train_1_3, X_test_1_3, Y_train, Y_test = train_test_split(X_1_3, Y, test_size=50, random_state=1)
X_train_4_6, X_test_4_6, Y_train, Y_test = train_test_split(X_4_6, Y, test_size=50, random_state=1)

lr_1_3 = LogisticRegression()
lr_4_6 = LogisticRegression()

lr_1_3.fit(X_train_1_3, Y_train)
lr_4_6.fit(X_train_4_6, Y_train)

Y_1_3_predict = lr_1_3.predict(X_test_1_3)
Y_4_6_predict = lr_4_6.predict(X_test_4_6)

Dif_1_3 = abs(Y_test - Y_1_3_predict)
Dif_4_6 = abs(Y_test - Y_4_6_predict)

stat = proportions_diff_z_stat_ind(Dif_1_3, Dif_4_6)
print "Не связанные",  proportions_diff_z_test(stat, 'two-sided')


print "Связанные ", proportions_diff_z_test(proportions_diff_z_stat_rel(Dif_1_3, Dif_4_6), 'two-sided')

Не связанные 0.00402223727206
Связанные  0.00329693845555


Task 4
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [11]:
print proportions_diff_confint_rel(Dif_1_3, Dif_4_6)

(0.059945206279614305, 0.30005479372038568)


Task 6

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [31]:
n = float(100)
sigma = 100
mu0 = float(525)
mu = float(541.4) 
z_stat = (mu- mu0) / (sigma/(n**0.5))

print z_stat
print 1 - scipy.stats.norm.cdf(z_stat)

1.64
0.0505025834741


Task 7

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [32]:
n = float(100)
sigma = 100
mu0 = float(525)
mu = float(541.5) 
z_stat = (mu- mu0) / (sigma/(n**0.5))

print z_stat
print 1 - scipy.stats.norm.cdf(z_stat)

1.65
0.0494714680336
